# CS 436
## Song Title Model

In [11]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import time

## Data Preparation

In [12]:
raw_data = pd.read_csv('tracks.csv')
raw_data = data.loc[:, ~data.columns.str.contains('^Unnamed')]

In [13]:
data = pd.DataFrame()
data['title'] = raw_data['title']
data

,title
0,Comedy
1,Ghost - Acoustic
2,Can't Help Falling In Love
3,Hold On
4,Hold On - Remix
...,...
38526,Fountain (I Am Good) - Live
38527,All My Life - Live
38528,At The Cross (Love Ran Red)
38529,How Can I Keep From Singing


In [14]:
titles = data.title.tolist()
words = ' '.join(titles)
words

'Comedy Ghost - Acoustic Can\'t Help Falling In Love Hold On Hold On - Remix Falling in Love at a Coffee Shop At My Worst Photograph All I Want For Christmas Is A Real Good Tan Party of One Party of One Party of One Lonely This Christmas Throwing Good After Bad This Time Tomorrow Lonely This Christmas The Haves When You\'re Wrong You and Me on the Rock You and Me on the Rock Speak Your Mind (From the Netflix Series "We The People") You and Me on the Rock You and Me on the Rock You and Me on the Rock You and Me on the Rock You and Me on the Rock A Drop in the Ocean So Far Away - Acoustic The Enemy Build It Better Sleepwalking in the Rain What\'s Left of You And I Love Her We Can\'t Stop Alone with You She Used To Be Mine In My Veins - Feat. Erin Mccarley A Drop In the Ocean Superman (It\'s Not Easy) When the End Comes You\'re Still The One Light Me Up The Long Road What Would I Do Without You Have It All Heavy Heart The House We Never Built Please Don\'t Say You Love Me Please Don\'t Sa

In [15]:
from nltk.tokenize import word_tokenize
from collections import Counter as ctr
import string

punctuation = string.punctuation

tokens = word_tokenize(words)
filtered_tokens = [word for word in tokens if word not in punctuation]
filtered_tokens = [word.lower() for word in filtered_tokens]

word_counts = ctr(filtered_tokens)
word_counts.most_common()
# christmas, remix, mix, remastered, acoustic

[('the', 9340),
 ('of', 3467),
 ('you', 2952),
 ('in', 2159),
 ('i', 2013),
 ('a', 1989),
 ('to', 1927),
 ('christmas', 1842),
 ('feat', 1676),
 ('me', 1667),
 ('it', 1605),
 ("'s", 1520),
 ('love', 1470),
 ('my', 1431),
 ('on', 1390),
 ('remix', 1347),
 ('from', 1282),
 ('for', 1268),
 ('with', 1197),
 ('version', 1068),
 ('and', 1063),
 ('is', 1049),
 ('mix', 945),
 ("''", 923),
 ("n't", 908),
 ('``', 891),
 ('remastered', 802),
 ('your', 756),
 ('live', 751),
 ('do', 717),
 ('all', 697),
 ('be', 667),
 ('this', 621),
 ('what', 586),
 ('we', 584),
 ('time', 567),
 ('song', 534),
 ('original', 509),
 ('edit', 506),
 ('one', 500),
 ('at', 493),
 ('heart', 472),
 ('that', 468),
 ('world', 463),
 ('night', 441),
 ('life', 434),
 ('up', 432),
 ('little', 426),
 ('no', 422),
 ('out', 421),
 ('let', 415),
 ('are', 375),
 ('remaster', 364),
 ('way', 343),
 ('baby', 341),
 ('when', 322),
 ('radio', 322),
 ('back', 321),
 ('know', 319),
 ('day', 310),
 ('down', 305),
 ('by', 299),
 ('single', 

In [37]:
keywords = ['Christmas', 'Remix', 'Mix', 'Remastered', 'Acoustic', '- Live', 'Snowman', 'Frosty']
filtered_tracks = data[data['title'].str.contains('|'.join(keywords))]

tracks = data.drop(index=filtered_tracks.index)
tracks

,title
0,Comedy
2,Can't Help Falling In Love
3,Hold On
5,Falling in Love at a Coffee Shop
6,At My Worst
...,...
38524,Come Holy Spirit
38525,Healing Begins
38528,At The Cross (Love Ran Red)
38529,How Can I Keep From Singing


In [38]:
terms = tracks.title.tolist()
text = ''
for t in terms:
    text=text+' ' +t
text = text[1::]

In [39]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

476 unique characters


In [40]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [41]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [42]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
sequences

<BatchDataset element_spec=TensorSpec(shape=(101,), dtype=tf.int32, name=None)>

In [43]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)
dataset

<MapDataset element_spec=(TensorSpec(shape=(100,), dtype=tf.int32, name=None), TensorSpec(shape=(100,), dtype=tf.int32, name=None))>

In [44]:
# Batch size
BATCH_SIZE = 10

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset element_spec=(TensorSpec(shape=(10, 100), dtype=tf.int32, name=None), TensorSpec(shape=(10, 100), dtype=tf.int32, name=None))>

In [45]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 100

# Number of RNN units
rnn_units = 100

In [46]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [47]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [48]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (10, None, 100)           47600     
                                                                 
 gru_3 (GRU)                 (10, None, 100)           60600     
                                                                 
 dense_3 (Dense)             (10, None, 476)           48076     
                                                                 
Total params: 156,276
Trainable params: 156,276
Non-trainable params: 0
_________________________________________________________________


In [49]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)


In [50]:
model.compile(optimizer='adam', loss=loss,run_eagerly=True)

In [51]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [52]:
EPOCHS = 50

In [53]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/50
723/723 [==============================] - 18s 23ms/step - loss: 2.8681
Epoch 2/50
723/723 [==============================] - 17s 23ms/step - loss: 2.1973
Epoch 3/50
723/723 [==============================] - 17s 23ms/step - loss: 2.0204
Epoch 4/50
723/723 [==============================] - 17s 23ms/step - loss: 1.9231
Epoch 5/50
723/723 [==============================] - 19s 26ms/step - loss: 1.8626
Epoch 6/50
723/723 [==============================] - 20s 27ms/step - loss: 1.8191
Epoch 7/50
723/723 [==============================] - 19s 25ms/step - loss: 1.7883
Epoch 8/50
723/723 [==============================] - 19s 25ms/step - loss: 1.7642
Epoch 9/50
723/723 [==============================] - 18s 24ms/step - loss: 1.7444
Epoch 10/50
723/723 [==============================] - 17s 23ms/step - loss: 1.7284
Epoch 11/50
723/723 [==============================] - 16s 22ms/step - loss: 1.7332
Epoch 12/50
723/723 [==============================] - 19s 26ms/step - loss: 1.7037
E

In [56]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [60]:
def generate_text(model, start_string,t):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 20

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = t

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [61]:
t_01 = (generate_text(model, start_string=u"Love Me Too ",t=0.1))
t_02 = (generate_text(model, start_string=u"Powerful ",t=0.2))
t_06 = (generate_text(model, start_string=u"What Is ",t=0.6))
t_08 = (generate_text(model, start_string=u"Hello ",t=0.8))

In [62]:
T = [0.1,0.2,0.6,0.8]
strings = [t_01,t_02,t_06,t_08]
for i in range(len(T)):
    print('Temperature = %.1f \n Strings Produced = %s'%(T[i],strings[i]))

Temperature = 0.1 
 Strings Produced = Love Me Too The Sun Stars The Be
Temperature = 0.2 
 Strings Produced = Powerful Man Waves Show The D
Temperature = 0.6 
 Strings Produced = What Is Common To Me Breathe
Temperature = 0.8 
 Strings Produced = Hello Way World) State Mis
